In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [2]:
data_path = 'data/train'
train_df = pd.read_csv(os.path.join(data_path, 'train_ratings.csv'))
train_df['watched'] = 1

In [11]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

train_df['user_id'] = user_encoder.fit_transform(train_df['user'])
train_df['item_id'] = item_encoder.fit_transform(train_df['item'])


In [5]:
def generate_negative_samples(train_df, num_items, max_negative_per_user=10, negative_ratio=0.5, random_seed=42):
    """
    네거티브 샘플 반환
    """
    np.random.seed(random_seed)

    users = train_df['user_id'].unique()
    positive_interactions = set(zip(train_df['user_id'], train_df['item_id']))
    all_items = set(range(num_items))  # Assuming item IDs are from 0 to num_items - 1
    
    negative_samples = []
    
    # 유저별로 진행
    for user in tqdm(users, desc="Generating negative samples", unit="user"):
        user_items = train_df[train_df['user_id'] == user]['item_id'].tolist()
        num_user_items = len(user_items)
        non_interacted_items = list(all_items - set(user_items))

        # 샘플링 조건
        if num_user_items <= 500:
            num_negative = int(num_user_items * negative_ratio)
        else:
            num_negative = max_negative_per_user

        # 네거티브 샘플링
        sampled_items = np.random.choice(non_interacted_items, size=num_negative, replace=False)
        for item in sampled_items:
            if (user, item) not in positive_interactions:
                negative_samples.append([user, item, 0])  # watched=0

    # 결과 데이터프레임 생성
    negative_df = pd.DataFrame(negative_samples, columns=['user_id', 'item_id', 'watched'])

    return negative_df

In [33]:
num_items = train_df['item_id'].nunique()
negative_df = generate_negative_samples(train_df, num_items, random_seed=42)


Generating negative samples: 100%|██████████| 31360/31360 [04:50<00:00, 108.07user/s]


In [13]:
negative_df['user'] = user_encoder.inverse_transform(negative_df['user_id'])
negative_df['item'] = item_encoder.inverse_transform(negative_df['item_id'])


In [14]:
# 기존 데이터와 병합
final_data = pd.concat([train_df, negative_df])
final_data.reset_index(drop=True, inplace=True)
final_data

,user,item,time,watched,user_id,item_id
0,11,4643,1.230783e+09,1,0,2505
1,11,170,1.230783e+09,1,0,109
2,11,531,1.230783e+09,1,0,319
3,11,616,1.230783e+09,1,0,368
4,11,2140,1.230783e+09,1,0,1183
...,...,...,...,...,...,...
7318594,138493,177,NaN,0,31359,115
7318595,138493,43832,NaN,0,31359,4863
7318596,138493,2791,NaN,0,31359,1547
7318597,138493,55267,NaN,0,31359,5328


In [15]:
class CDAEModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=128, dropout_rate=0.1):
        super(CDAEModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.hidden_layer1 = nn.Linear(embedding_dim, 1024)
        self.hidden_layer2 = nn.Linear(1024, embedding_dim)
        self.output_layer = nn.Linear(embedding_dim, num_items)
        self.dropout = nn.Dropout(dropout_rate)
        self.leaky_relu = nn.LeakyReLU()

    def forward(self, user_input, item_input):
        user_embedded = self.user_embedding(user_input).squeeze(1)  # (batch_size, embedding_dim)
        item_embedded = self.item_embedding(item_input).squeeze(1)
        
        hidden = self.leaky_relu(self.hidden_layer1(item_embedded))
        hidden = self.leaky_relu(self.hidden_layer2(hidden))
        
        combined = user_embedded + hidden
        # combined = torch.cat([user_embedded, hidden], dim=1)
        # combined = self.leaky_relu(self.fc_combined(combined))  # Optional FC layer

        output = torch.sigmoid(self.output_layer(combined))
        return output

In [16]:
# CDAE 모델 인스턴스 생성
num_users = train_df['user_id'].nunique()
num_items = train_df['item_id'].nunique()
model = CDAEModel(num_users, num_items, embedding_dim=128)


In [17]:
class InteractionDataset(Dataset):
    def __init__(self, data, num_items):
        """
        data: DataFrame with columns ['user_id', 'item_id', 'watched']
        num_items: Total number of unique items
        """
        self.data = data
        self.num_items = num_items

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        user_id = row['user_id']
        item_id = int(row['item_id'])
        
        label = torch.zeros(self.num_items)
        label[item_id] = row['watched']  # 1 for positive, 0 for negative
        
        return user_id, item_id, label

In [10]:
def save_checkpoint(model, optimizer, epoch, filepath="model_checkpoint.pth"):
    """
    Save the model's state and optimizer state for resuming training.
    """
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
    }
    torch.save(checkpoint, filepath)
    print(f"Checkpoint saved to {filepath}")


In [13]:
def train_model(model, train_data, num_items, num_epochs=10, batch_size=128, learning_rate=0.001, device='cuda'):
    """
    Train the CDAE model.
    Args:
        model: CDAE model instance
        train_data: DataFrame with columns ['user_id', 'item_id', 'watched']
        num_items: Total number of unique items
        num_epochs: Number of training epochs
        batch_size: Batch size for DataLoader
        learning_rate: Learning rate for optimizer
        device: Device to train on ('cuda' or 'cpu')
    """
    # Prepare dataset and dataloader
    dataset = InteractionDataset(train_data, num_items)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Move model to device
    model.to(device)

    # Training loop
    model.train()
    try:
        for epoch in range(num_epochs):
            epoch_loss = 0.0
            with tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as t:
                for user_input, item_input, labels in t:
                    user_input = user_input.to(device).long()
                    item_input = item_input.to(device).long()
                    labels = labels.to(device)

                    # Forward pass
                    outputs = model(user_input, item_input)

                    # Compute loss
                    loss = criterion(outputs, labels)
                    epoch_loss += loss.item()

                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    t.set_postfix(loss=loss.item())

            print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(dataloader):.5f}")

    except KeyboardInterrupt:
        print("Training interrupted.")
        save_checkpoint(model, optimizer, epoch + 1, "model_checkpoint.pth")
        return model, optimizer, epoch
    
    print("Training complete!")
    save_checkpoint(model, optimizer, epoch + 1, "model_checkpoint.pth")
    return model, optimizer, epoch

In [25]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
trained_model, optimizer, last_epoch = train_model(model, final_data, num_items, num_epochs=10, batch_size=128, learning_rate=0.001, device=device)


Using device: cuda


Epoch 1/10: 100%|██████████| 57177/57177 [15:19<00:00, 62.20batch/s, loss=7.57e-5] 


Epoch 1 Loss: 0.00026


Epoch 2/10: 100%|██████████| 57177/57177 [15:35<00:00, 61.15batch/s, loss=6.48e-5] 


Epoch 2 Loss: 0.00008


...

Epoch 3 Loss: 0.00007


Epoch 4/10: 100%|██████████| 57177/57177 [15:21<00:00, 62.03batch/s, loss=8.91e-5] 


Epoch 4 Loss: 0.00006


Epoch 5/10:   4%|▍         | 2174/57177 [00:36<15:13, 60.24batch/s, loss=7.73e-5]


Training interrupted.
Checkpoint saved to model_checkpoint.pth


In [18]:
def load_checkpoint(filepath, model, optimizer=None, device='cuda'):
    """
    Load the model and optimizer state from a checkpoint for resuming training.
    """
    checkpoint = torch.load(filepath, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print("Model state loaded.")
    
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Optimizer state loaded.")
    
    epoch = checkpoint['epoch']
    print(f"Resuming from epoch {epoch}")
    return epoch


In [ ]:
def continue_training(model, train_data, num_items, checkpoint_path, num_epochs=10, batch_size=128, learning_rate=0.001, device='cuda'):
    """
    Continue training from a saved checkpoint.
    Args:
        model: CDAE model instance.
        train_data: DataFrame with training data.
        num_items: Total number of unique items.
        checkpoint_path: Path to the saved checkpoint.
        num_epochs: Total number of epochs to train.
        batch_size: Batch size for DataLoader.
        learning_rate: Learning rate for optimizer.
        device: Device to train on ('cuda' or 'cpu').
    """
    # Prepare dataset and dataloader
    dataset = InteractionDataset(train_data, num_items)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Load checkpoint
    start_epoch = 0
    if checkpoint_path:
        start_epoch = load_checkpoint(checkpoint_path, model, optimizer, device)

    # Move model to device
    model.to(device)

    # Training loop
    model.train()
    try:
        for epoch in range(start_epoch, num_epochs):
            epoch_loss = 0.0
            with tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as t:
                for user_input, item_input, labels in t:
                    user_input = user_input.to(device).long()
                    item_input = item_input.to(device).float()
                    labels = labels.to(device)

                    # Forward pass
                    outputs = model(user_input, item_input)

                    # Compute loss
                    loss = criterion(outputs, labels)
                    epoch_loss += loss.item()

                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    t.set_postfix(loss=loss.item())

            print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(dataloader):.4f}")

    except KeyboardInterrupt:
        print("Training interrupted.")
        save_checkpoint(model, optimizer, epoch + 1, f"model_checkpoint{epoch + 1}.pth")
        return model, optimizer, epoch
    
    print("Training complete!")
    save_checkpoint(model, optimizer, epoch + 1, f"model_checkpoint{epoch + 1}.pth")
    return model, optimizer, epoch


In [ ]:
# 재학습용
new_model = CDAEModel(num_users, num_items, embedding_dim=128)

model = continue_training(new_model, final_data, num_items, checkpoint_path="model_checkpoint3.pth", num_epochs=20, batch_size=128, learning_rate=0.001)


모델 로드

In [19]:
model = CDAEModel(num_users, num_items, embedding_dim=128)

# 저장된 체크포인트에서 모델 불러오기 (체크포인트 이름 확인!)
load_checkpoint("model_checkpoint.pth", model, device='cuda')


Model state loaded.
Resuming from epoch 5


5

추천

In [21]:
def recommend_top_k(model, final_data, num_users, k=10, device='cuda'):
    """
    Recommend top K items for each user based on the trained model.
    """
    model.eval()
    model.to(device)
    all_recommendations = {}
    all_items = set(final_data['item_id'].unique())  # 전체 아이템 집합

    with tqdm(total=num_users, desc="Recommending", unit="user") as pbar:
        for user_id in range(num_users):
            # 유저가 상호작용한 아이템
            interacted_items = final_data[(final_data['user_id'] == user_id) & (final_data['watched'] == 1)]['item_id'].tolist()
            
            # 추천 후보 아이템: 상호작용하지 않은 전체 아이템
            candidates = list(all_items - set(interacted_items))

            # 유저 ID와 후보 아이템 ID를 텐서로 변환
            user_input = torch.tensor([user_id], dtype=torch.long, device=device)
            user_embedded = model.user_embedding(user_input).squeeze(0)  # (embedding_dim,)

            # 후보 아이템 임베딩 계산
            candidate_input = torch.tensor(candidates, dtype=torch.long, device=device)
            candidate_embedded = model.item_embedding(candidate_input)  # (num_candidates, embedding_dim)
            
            # 점수 계산 (유저-아이템 내적)
            with torch.no_grad():
                scores = torch.matmul(candidate_embedded, user_embedded)  # (len(candidates),)

             # Top K 추천
            top_k_indices = torch.topk(scores, k=min(k, len(candidates))).indices.cpu().numpy()
            top_k_items = [candidates[idx] for idx in top_k_indices]

            # 결과 저장
            all_recommendations[user_id] = top_k_items[:k]
            pbar.update(1)

    return all_recommendations

In [22]:
recommendations = recommend_top_k(model, final_data, num_users, num_items, k=10)


Recommending: 100%|██████████| 31360/31360 [16:01<00:00, 32.63user/s]


In [26]:
for user_id, item_ids in recommendations.items():
    if len(item_ids) != 10:
        print(f"User {user_id} has {len(item_ids)} recommendations instead of 10.")

In [29]:
# CSV 파일에 저장할 때, 처음에는 헤더를 포함하고, 이후에는 추가로 저장
header = True
chunk_size = 1000  # 한 번에 처리할 사용자 수

rows = []
for i, (user_id, item_ids) in enumerate(recommendations.items()):
    original_user_id = user_encoder.inverse_transform([user_id])[0]
    original_item_ids = item_encoder.inverse_transform(item_ids)
    for item_id in original_item_ids:
        rows.append([original_user_id, item_id])
    
    # 청크 크기만큼 처리한 후 저장
    if (i + 1) % chunk_size == 0 or i == len(recommendations) - 1:
        df = pd.DataFrame(rows, columns=["user", "item"])
        df.to_csv('cdae_rec3.csv', mode='w', header=header, index=False)
        header = False  # 이후에는 헤더를 포함하지 않음
        rows = []  # 저장 후 리스트 초기화

print("추천 결과가 저장되었습니다.")

추천 결과가 저장되었습니다.
